In [27]:
import networkx as nx

In [179]:
class Top:
    ''' A class for representing a finite topological space. 
        ------------
        Attributes |
        ----------------------------------------------------------------
        points: a set {x | x in X } of the points of the top. space
                The elements should all be hashable

        opens: a dictionary of the basic open sets. It is of the form
        { x : { y | x <= y} } where x <= y means x in closure({y})

        closures:   dictionary of the closures of each point
                    { x : cl({x}) }
                    Starts as empty because computing all is O(n^2)
                    Run getClosures() method if all are needed. Else,
                    just call X.cl(x) or X[x] for closure of 1 point
        ----------------------------------------------------------------
    
    '''
    def __init__(self, data, discrete=False):
        # can pass a networkx graph as the data
        if type(data) == nx.DiGraph:
            opens = {}
            for node in data.nodes:
                opens[node] = nx.descendants(data,node) | {node}
            self.opens = opens
            self.points = set(data.nodes)

        elif type(data) == dict:
            self.opens = data
            self.points = set(data)
        
        elif type(data) == int:
            assert data >= 0, 'Topology of integer can\'t be negative'
            self.points = {n for n in range(data)}            
            if not discrete: # order linearly
                self.opens = {n: {k for  k in range(n+1)} 
                              for n in range(data)}
            else: # discrete topology
                self.opens = {n: {n} for n in range(data)}
            
        self.closures = {} # can specify closures and ordering
        self.ordering = None # but left empty until needed


    def __repr__(self):
        return str(self.opens)
    
    def __len__(self):
        return len(self.points)
    
    def __contains__(self, x):
        return x in self.points
    
    def __iter__(self):
        return iter(self.points)

    def __call__(self, x):
        # Top(x) returns the same as self.opens[x] = {y | y <= x}
        return self.opens[x]
        
    def __getitem__(self, x):
        # Top[x] returns the same as Top.cl(x) or Top.closures[x]
        # This convention mirrors the notation (a,b) for open interval 
        # and [a,b] for closed interval
        try:
            return self.closures[x]
        except:
            self.closures[x] = self.cl(x)
            return self.closures[x]
        
    def __mul__(self, other):
        # product topology
        return self.product(other)
    
    def __add__(self, other):
        # disjoint union topology
        if not self.points & other.points: # already disjoint
            return Top(self.opens | other.opens)
        else: # force them to be disjoint
            point0 = Top({0: {0}})
            point1 = Top({1: {1}})
            return Top((point0 * self).opens | (point1 * other).opens)
        
    def __eq__(self, other):
        return self.opens == other.opens
    
    def __le__(self, other):
        # is a subspace
        if self.points & other.points != self.points:
            return False # isn't a subset
        for point in self.points:
            if other(point) & self.points != self(point):
                return False # not the same open sets
        return True
    
    def __lt__(self, other):
        # proper subspace
        return (not self == other) and self <= other
    
    def __ge__(self, other):
        # contains other as a subspace
        return other <= self
    
    def __gt__(self, other):
        # proper super space
        return (not self == other) and self >= other
    
    def __truediv__(self, subspace):
        # quotient by a subspace
        assert self >= subspace, 'Quotient defined for a subspace'
        quotient = set(self.points - subspace.points)

        # the quotient projection
        def pi(x, pt): 
            if x in self.points - subspace.points:
                return x
            elif x in subspace.points:
                return pt
            else:
                raise ValueError('projection map domain error')
        
        # adjoin a new point not in X \ A, make sure key isn't taken
        pt = '*'; idx = 0
        while pt in quotient:
            idx += 1
            pt = '*' + str(idx)
        # now make the open sets for each x in X \ A
        opens = {x: {pi(y, pt) for y in self(x)} for x in quotient}
        # add in the open set around the new extra point 
        opens[pt] = set()
        for a in subspace.points:
            opens[pt] |= {pi(x, pt) for x in self(a)}
        return Top(opens)
    
    def __pow__(self, other):
        return self.hom(other)

    def __invert__(self):
        return self.op()

    def cl(self, x):
        # returns the closure of the singleton {x}
        assert x in self.points,\
        f'{x} is not a point in the topological space'
        close = {y for y in self if x in self(y)}
        # close = { y | x <= y } = { y | x in self.opens[y] }
        return close
        
    def getClosures(self):
        # populates Top.closures for all elements
        for x in self:
            try: # don't bother if it's already computed
                self.closures[x]
            except:
                self.closures[x] = self.cl(x)

    def isleq(self, x, y):
        # partial order defined by x<=y iff x in self.opens[y]
        # or equivalently iff y in self.closures[x]
        return x in self(y)
    
    def isT0(self):
        # True iff self.opens[x] = self.opens[y] implies x = y
        checked = set()
        for x in self:
            for y in self.points - (checked | {x}):
                if self(x) == self(y):
                    return False
            checked |= {x} # don't check U_x=U_y and U_y=U_x separately
        return True
    
    def isT1(self):
        # for finite spaces, this is the same as being discrete
        for x in self:
            if self(x) != {x}:
                return False
        return True
    
    def subspace(self, subset):
        # subspace topology
        assert subset & self.points == subset, 'needs to be a subset'
        return Top({x: self(x) & subset for x in subset})
    
    def product(self, other):
        # cartesian product
        def prod(set1, set2):
            return {(x, y) for x in set1 for y in set2}
        points = prod(self.points, other.points)
        opens = {point : prod(self(point[0]), other(point[1])) \
                 for point in points}
        return Top(opens)
    
    def op(self):
        # reverse the ordering
        self.getClosures()
        op_opens = self.closures
        X_op = Top(op_opens)
        X_op.closures = self.opens
        return (X_op)
    
    def order(self, update=True):
        # returns a topological ordering of self.points, meaning that if
        # x <= y, then x will appear first in the list (but converse may 
        # be false if x and y aren't comparable)

        nodes = {x: {'start':0, 'end':0, 'parent':None, 'visited':False}
                 for x in self.points} # nodes for depth first search
        time = 0 
        def dfsVisit(pt): # one iteration of depth first search
            nonlocal time
            time += 1
            node = nodes[pt]
            node['start'] = time
            node['visited'] = True

            for child in self[pt]: # pt <= child
                childnode = nodes[child]
                if childnode['visited'] == False:
                    childnode['parent'] = node
                    dfsVisit(child)
            time += 1
            node['end'] = time
            node['visited'] = 'done'

        # now iterate over all points 
        for pt in self:
            if nodes[pt]['visited'] == False:
                dfsVisit(pt)

        # after depth first search, order the points by finish time
        top_order = sorted(nodes, reverse=True,
                           key=lambda x: nodes[x]['end'] )
        if update:
            self.ordering = top_order
        return tuple(top_order)
    
    def to(self, other, inj=False):
        ''' returns a list of continuous fnunctions self -> other. Each
        function is represented as a dictionary {x: f(x)}.
        X.to(Y) returns [{x: f(x)} for f: X --> Y continuous]'''
        P_ordered = self.order()          # topologically sorted points of P
        Q_points = list(other.points)      # points of Q to map into
        results = []

        # helper function to iteratively modify an existing function
        def backtrack(index, current_map):  
            if index == len(P_ordered): # complete function
                results.append(current_map.copy())
                return 
            x = P_ordered[index]
            for q in other.points: # potential values for f(x)
                valid = True # make sure (x <= y) ==> ( f(x) <= f(y) )
                for y in current_map:
                    fy = current_map[y]
                    if y in self.opens[x] and fy not in other.opens[q]:
                        valid = False
                        break
                if valid:
                    current_map[x] = q # assign f(x) = q
                    backtrack(index + 1, current_map) # progress 1 step
                    # reached end of valid mapping
                    del current_map[x] # remove last assignment, iterate

        backtrack(0, {})
        return results
    
    def hom(self, other, domain_order=False):
        # X.hom(Y) returns the topological space Hom(X, Y) with the com-
        # pact-open topology: f <= g iff f(x) <= g(x) for all x
        funcs = self.to(other)
        opens = {}
        order = self.ordering if self.ordering else self.order()
        def funcTuple(func):
            # use top ordering to write func as (f(x_0), ..., f(x_n))
            # so that it is hashable            
            return tuple([func[x] for x in order])

        func_set = [funcTuple for f in funcs]

        for f1 in funcs:
            key = funcTuple(f1)
            opens[key] = {funcTuple(f2) for f2 in funcs
                          if all(f2[x] in other.opens[f1[x]] 
                                        for x in self.points)    
                            }
        if domain_order:
            return order, Top(opens)
        return Top(opens)
    
    def relabel(self):
        # returns a homeomorphic top space with integer labels
        order = self.ordering if self.ordering else self.order()
        opens = {n:{m for m in range(len(self)) 
                    if order[m] in self(order[n])}
                for n in range(len(self))}
        return Top(opens)


In [180]:
X = Top({0:{0,1,3}, 1:{1}, 2:{1,2,3}, 3:{3}})
A = X.subspace({0,3})
X.getClosures()
X

{0: {0, 1, 3}, 1: {1}, 2: {1, 2, 3}, 3: {3}}

In [183]:
((Top(2) ** Top(2)) ** X).relabel()

{0: {0}, 1: {0, 1}, 2: {0, 1, 2}, 3: {3}, 4: {3, 4}, 5: {3, 5}, 6: {3, 4, 6}, 7: {3, 5, 7}, 8: {0, 1, 2, 3, 4, 6, 8}, 9: {0, 9}, 10: {0, 9, 10}, 11: {0, 3, 5, 7, 9, 10, 11}}

In [169]:
for i in enumerate(('a','b','c')):
    print(i)

(0, 'a')
(1, 'b')
(2, 'c')


In [31]:
a = {1:'a', 2:'d', 0:'c'}
sorted(a, key=lambda x: a[x])

[1, 0, 2]

In [32]:
d = {'a': {'a':1, 'b':2, 'c':3}}
d['a']['b']

2

In [33]:
def enumerate_order_preserving_maps1(P: Top, Q: Top):
    """
    Enumerates all order-preserving functions f: P → Q
    where P, Q are Top instances (finite topological spaces).
    """
    P_ordered = P.order()          # topologically sorted points of P
    Q_points = list(Q.points)      # points of Q to map into
    results = []

    def backtrack(index, current_map):
        if index == len(P_ordered):
            results.append(current_map.copy())
            return
        x = P_ordered[index]
        for q in Q_points:
            valid = True
            for y in current_map:
                fy = current_map[y]
                if P.isleq(y, x) and not Q.isleq(fy, q):
                    valid = False
                    break
            if valid:
                current_map[x] = q
                backtrack(index + 1, current_map)
                del current_map[x]

    backtrack(0, {})
    return results


In [34]:
def enumerate_order_preserving_maps2(P: Top, Q: Top):
    """
    Enumerates all order-preserving functions f: P → Q
    where P and Q are instances of the Top class.
    """
    P_ordered = P.order()         # topologically sorted points of P
    Q_points = list(Q.points)     # codomain options
    results = []

    def backtrack(index, current_map):
        if index == len(P_ordered):
            results.append(current_map.copy())
            return
        x = P_ordered[index]
        for q in Q_points:
            # Only check f(y) <= f(x) for y <= x (y has already been mapped)
            if all(not P.isleq(y, x) or Q.isleq(current_map[y], q)
                   for y in current_map):
                current_map[x] = q
                backtrack(index + 1, current_map)
                del current_map[x]

    backtrack(0, {})
    return results

In [39]:
def enumerate_order_preserving_maps3(P: Top, Q: Top):
    P_ordered = P.order()
    Q_ordered = Q.order()  # topological order of Q
    results = []

    def backtrack(index, current_map):
        if index == len(P_ordered):
            results.append(current_map.copy())
            return

        x = P_ordered[index]
        for q in Q_ordered:
            if all(not P.isleq(y, x) or current_map[y] in Q.opens[q]
                   for y in current_map):
                current_map[x] = q
                backtrack(index + 1, current_map)
                del current_map[x]

    backtrack(0, {})
    return results

In [61]:
enumerate_order_preserving_maps1(X, Top(2))

[{3: 0, 1: 0, 2: 0, 0: 0},
 {3: 0, 1: 0, 2: 0, 0: 1},
 {3: 0, 1: 0, 2: 1, 0: 0},
 {3: 0, 1: 0, 2: 1, 0: 1},
 {3: 0, 1: 1, 2: 1, 0: 1},
 {3: 1, 1: 0, 2: 1, 0: 1},
 {3: 1, 1: 1, 2: 1, 0: 1}]

In [ ]:
def enumerate_order_preserving_maps4(P: Top, Q: Top):
    """
    Enumerates all order-preserving functions f: P → Q
    where P, Q are Top instances (finite topological spaces).
    """
    P_ordered = P.order()          # topologically sorted points of P
    Q_points = list(Q.points)      # points of Q to map into
    results = []

    def backtrack(index, current_map):
        if index == len(P_ordered):
            results.append(current_map.copy())
            return
        x = P_ordered[index]
        for q in Q.points:
            valid = True
            for y in current_map:
                fy = current_map[y]
                if y in P.opens[x] and fy not in Q.opens[q]:
                    valid = False
                    break
            if valid:
                current_map[x] = q
                backtrack(index + 1, current_map)
                del current_map[x]

    backtrack(0, {})
    return results

In [74]:
%timeit enumerate_order_preserving_maps1(Top(5), X*X*X)

4.13 s ± 304 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [88]:
enumerate_order_preserving_maps4(Top(2), Top(3))

[{0: 0, 1: 0},
 {0: 0, 1: 1},
 {0: 0, 1: 2},
 {0: 1, 1: 1},
 {0: 1, 1: 2},
 {0: 2, 1: 2}]